In [ ]:
import sys
import os
sys.path.insert(0, './../../Research')

import onnxruntime

from PIL import Image
import numpy as np
from scipy.special import softmax
from tqdm.notebook import tqdm

import torch
import torch.nn.functional as F

import torchvision.transforms as transforms
from time import time

from utils import get_classes

In [ ]:
def to_numpy(tensor):
    """Tensor to numpy array"""
    
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

## Model path and holdout images folder

In [ ]:
# ONNX model
model_path = './../../Models/fashionNet.onnx'
classes = get_classes()

# Folder of images
holdout_folder = "./../data/fashionmnist/images/"

In [ ]:
# ONNXruntime engine

ort_session = onnxruntime.InferenceSession(model_path)

## Inference

In [ ]:
correct_predictions = 0
total_predictions = 0

In [ ]:
time_per_image = []
time_start = time()

# Inferencing Images one by one
for images in tqdm(os.listdir(holdout_folder)):
    if not images.endswith('.pgm'):
        continue
    time_img_st = time()
    img_name = os.path.join(holdout_folder, images)
    img_label = classes[int(os.path.basename(img_name).split('_')[0])]

    img_PIL = Image.open(img_name)
    img = transforms.ToTensor()(img_PIL)
    img = img.unsqueeze(0)
    
    # compute ONNX Runtime output prediction
    ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img)}
    ort_outs = torch.tensor(ort_session.run(None, ort_inputs), dtype=torch.float32)
    
    ort_outs_tensor = ort_outs[0].squeeze(0)

    preds = torch.argmax(ort_outs_tensor, dim=0)
    pred_label = classes[preds]
    
    total_predictions += 1
    if pred_label == img_label:
        correct_predictions += 1
    time_img_en = time()
    time_per_image.append(time_img_en - time_img_st)
time_end = time()

## Printing stats

In [ ]:
print("Accuracy         = ", correct_predictions*100/total_predictions)
print("Total time (sec) = ", time_end - time_start)
print("Latency          = ", np.mean(time_per_image))

## Stats from my Run

In [ ]:
# FashionNet

# Accuracy         =  92.33844103930713
# Total time (sec) =  4.945388555526733
# Latency          =  0.001576131856258832

In [ ]:
# torNet

# Accuracy         =  91.87208527648235
# Total time (sec) =  29.028265476226807
# Latency          =  0.009158259149712774

In [ ]:
# efficientNet

# Accuracy         =  89.44037308461026
# Total time (sec) =  48.72265887260437
# Latency          =  0.01540562941343446

In [ ]:
## TensorRT

In [ ]:
# torNet

# Percentage Accuracy = 91.8721
# Total time taken   = 48.842
# Latency            = 0.0162661

In [ ]:
# FashionCNN

# Percentage Accuracy = 92.3384
# Total time taken   = 2.8788
# Latency            = 0.000956716

In [ ]:
# Efficient Net

#  Unable to run due to the following error
'''
terminate called after throwing an instance of 'std::out_of_range'
  what():  Attribute not found: pads
Aborted (core dumped)
root@0c209c19480f:/data/Inference/bin# clear
'''